In [1]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import geopandas as gpd
# en la carpeta Priorizacion_funciones está el archivo analisis_geo.py
from Priorizacion_funciones.analisis_geo import *
import os
os.getcwd()

'C:\\Users\\DarioAlejandroLatorr\\OneDrive\\claro_20_abril_2021\\ENVIO_DARIO\\MODELO_PRIORIZACION_DATA_ORIGEN_ADRIANA'

In [4]:
import pysal
from pysal.lib import weights
from pysal.explore.esda import Moran, Moran_Local
import splot
from splot.esda import moran_scatterplot, plot_moran, lisa_cluster

# Análisis Geo General

In [4]:
var_int = 'SCORE'
df_var = pd.read_csv('Data_priorizacion/priorizacion.csv',
                             usecols=['NODO',var_int])
df_var.drop_duplicates(inplace=True)

## Correr sólo uno de los siguientes dos bloques de código

Correr el siguiente bloque para generar la capa de unión, el proceso puede llevar más de media hora.

In [5]:
mgn_rur_layer = gpd.read_file('Data_geo\MGN\MGN_RUR_SECTOR.shp')
hfc_layer = gpd.read_file('Data_geo\HFC\HFC_UNIFICADO_MARZO_WGS.shp')
hfc_rur_union =  preparar_capa(mgn_rur_layer, hfc_layer)
df_lisa = hfc_rur_union.merge(df_var, how ='left', on='NODO')

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
El proceso puede tardar más de media hora.
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
--------------------------------------------------------------------------------
Tiempo de inicio del proceso: 2021-07-15 11:41
--------------------------------------------------------------------------------
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Se eliminan los siguientes nodos por no cumplir el límite de desviación
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
      Longitud   Latitud       max
NODO                              
CHF   0.015961  0.008103  0.015961
OAC   0.405627  0.166212  0.405627
OVM   0.435211  0.249230  0.435211
--------------------------------------------------------------------------------
Uniendo las capas...
---------------------------------------------------------------------------

KeyboardInterrupt: 

#### Si la capa en Data_geo\MUNICIPIOS_ANALISIS\layer_analisis.shp existe y es reciente se puede usar esa capa para el análisis, si es así y no se corrío el anterior bloque, correr el siguiente bloque de código.

In [6]:
mgn_rur_layer = gpd.read_file('Data_geo\MGN\MGN_RUR_SECTOR.shp')
hfc_rur_union = gpd.read_file('Data_geo\MUNICIPIOS_ANALISIS\layer_analisis.shp')
df_lisa = hfc_rur_union.merge(df_var, how ='left', on='NODO')

#### Seguir con la preparación del dataframe

In [7]:
df_lisa = preparar_lisa(df_lisa, var_int) 

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 23609 entries, 0 to 23608
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   NODO            23609 non-null  object  
 1   NOMBRE          15431 non-null  object  
 2   CIUDAD          14914 non-null  object  
 3   Longitud        15446 non-null  float64 
 4   Latitud         15446 non-null  float64 
 5   DPTO_CCDGO      22931 non-null  object  
 6   MPIO_CCDGO      22931 non-null  object  
 7   CLAS_CCDGO      22931 non-null  object  
 8   SETR_CCDGO      22931 non-null  object  
 9   SETR_CCNCT      22931 non-null  object  
 10  SETR_NAREA      22931 non-null  float64 
 11  SETR_CSMBL      22931 non-null  object  
 12  SETR_NANO       22931 non-null  float64 
 13  COD_MPIO        22931 non-null  object  
 14  SHAPE_AREA      22931 non-null  float64 
 15  SHAPE_LEN       22931 non-null  float64 
 16  geometry        23609 non-null  geometry
 17  SCOR

C:\Users\DarioAlejandroLatorr\anaconda3\envs\ds_sqdm_claro\lib\site-packages\mapclassify\classifiers.py:234: UserWarning: Warning: Not enough unique values in array to form k classes
  Warn(
C:\Users\DarioAlejandroLatorr\anaconda3\envs\ds_sqdm_claro\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: setting k to 2
  Warn("Warning: setting k to %d" % k_q, UserWarning)


In [8]:
w, df_lisa = get_weights(df_lisa)
y = df_lisa[var_int]
df_lisa["w_var"] = weights.lag_spatial(w, y)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
El proceso puede tardar más de 5 minutos.
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


C:\Users\DarioAlejandroLatorr\anaconda3\envs\ds_sqdm_claro\lib\site-packages\pysal\lib\weights\weights.py:167: UserWarning: There are 7 disconnected observations 
  Island ids: 110011000-ICC, 110011000-IMG, 110011000-INF, 110011000-MHT1M, 880013024-SIN_NODO, 880013025-SIN_NODO, 885643001-SIN_NODO
  warnings.warn("There are %d disconnected observations" % ni + ' \n '


('WARNING: ', '110011000-ICC', ' is an island (no neighbors)')
('WARNING: ', '110011000-IMG', ' is an island (no neighbors)')
('WARNING: ', '110011000-MHT1M', ' is an island (no neighbors)')
('WARNING: ', '110011000-INF', ' is an island (no neighbors)')
('WARNING: ', '880013025-SIN_NODO', ' is an island (no neighbors)')
('WARNING: ', '880013024-SIN_NODO', ' is an island (no neighbors)')
('WARNING: ', '885643001-SIN_NODO', ' is an island (no neighbors)')
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Pesos obetnidos e islas eliminadas del dataframe para correr el Moran Local.
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


C:\Users\DarioAlejandroLatorr\anaconda3\envs\ds_sqdm_claro\lib\site-packages\pysal\lib\weights\weights.py:170: UserWarning: The weights matrix is not fully connected. There are 4 components
  warnings.warn("The weights matrix is not fully connected. There are %d components" % self.n_components)


In [9]:
moran_loc = Moran_Local(y, w)
df_lisa['SCORE_ESPACIAL'] = ((moran_loc.q ==1) * 1)*(1-2*moran_loc.p_sim)*df_lisa['SCORE']

In [10]:
df_priorizacion = pd.read_csv('Data_priorizacion/priorizacion.csv')
print(df_priorizacion['EST_GLOB_FIN'].value_counts())
print(df_priorizacion['EST_GLOB_INI'].value_counts())

AMARILLO    7389
VERDE       5765
ROJO        2487
Name: EST_GLOB_FIN, dtype: int64
AMARILLO    7471
VERDE       6306
ROJO        1864
Name: EST_GLOB_INI, dtype: int64


In [11]:
df_priorizacion = pd.read_csv('Data_priorizacion/priorizacion.csv')
df_lisa = df_lisa[['NODO', 'Latitud','Longitud','SCORE_ESPACIAL']].copy()
dic_agg = {'Latitud':'mean','Longitud':'mean', 'SCORE_ESPACIAL':'mean'}
df_lisa = df_lisa.groupby('NODO').agg(dic_agg).reset_index()
df_priorizacion = df_priorizacion.merge(df_lisa, how='left', on='NODO')
dic_semaforo = {'VERDE':'1_VERDE','AMARILLO':'2_AMARILLO','ROJO':'3_ROJO'}
df_priorizacion['SEMAFORO_FIN'] = [dic_semaforo[x] for x in df_priorizacion['EST_GLOB_FIN']]
df_priorizacion['SEMAFORO_INI'] = [dic_semaforo[x] for x in df_priorizacion['EST_GLOB_INI']]
df_priorizacion.to_csv('Data_priorizacion/priorizacion_dash_excel.csv'
                       ,index=False, sep=';', decimal=',')
df_priorizacion.to_csv('Data_priorizacion/priorizacion_dash.csv')
print(df_priorizacion['SEMAFORO_FIN'].value_counts())
print(df_priorizacion['SEMAFORO_INI'].value_counts())

2_AMARILLO    7389
1_VERDE       5765
3_ROJO        2487
Name: SEMAFORO_FIN, dtype: int64
2_AMARILLO    7471
1_VERDE       6306
3_ROJO        1864
Name: SEMAFORO_INI, dtype: int64
